<h2>Fetch pages by asyncio</h2>

In [9]:
import nest_asyncio
nest_asyncio.apply()

In [10]:
import asyncio
import aiohttp
from tqdm.notebook import tqdm

In [11]:
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd

In [12]:
from datetime import datetime

def getNowTime():
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt

# dictionary with {page: content} structure
dictContent = {}

# column text according to headers
def colText(_soup: bs, _headers: str)->list:
    lst = []
    all_text = _soup.findAll("td", attrs={'headers': _headers})    
    for txt in all_text:
        num = txt.get("rowspan")
        if num == None: # no rowspan
            lst.append(txt.text)
            continue        
        if num == "1":
            lst.append(txt.text)
            continue
        if num != "1":
            for j in range(int(num)):
                lst.append(txt.text)
    return lst

def pageContent(_content: str)-> dict:
    if _content == "":
        print("Empty content!")
        return {}
    soup = bs(_content, 'html.parser')
    lstReg = colText(soup, 'reg')
    lstName = colText(soup, 'name')
    lstDate = colText(soup, 'date')
    lstQualification = colText(soup, 'qualification')
    lstQuali = [x for (i, x) in enumerate(lstQualification) if i % 2 == 0]
    lstYear = [x for (i, x) in enumerate(lstQualification) if i % 2 != 0]
    myDict = {'regno': lstReg, 'name': lstName, 'address': lstDate, 'qualification': lstQuali, 'year': lstYear}
    return myDict

async def fetch_page(_page: int):
    async with aiohttp.ClientSession() as session:
        link = f'https://www.mchk.org.hk/tc_chi/list_register/list.php?page={_page}&ipp=20&type=L'
        async with session.get(link) as response:
            content = await response.text()
            dictContent[_page] = pageContent(content)
            print(_page, " is fetched at: ", getNowTime())

def run_fetching(_lastpage: int):
    #_url = 'https://www.w3schools.com/css'
    #image_urls = [ f"{_url}/img_5terre.jpg", f"{_url}/img_forest.jpg", f"{_url}/img_lights.jpg", f"{_url}/img_mountains.jpg" ]
    tasks = [ fetch_page(page) for page in range(1, _lastpage+1)]
    asyncio.get_event_loop().run_until_complete(asyncio.gather(*tasks))
    print(f"All {len(dictContent)} pages are fetched!")

In [13]:
run_fetching(4)

3  is fetched at:  26/08/2024 19:09:15
1  is fetched at:  26/08/2024 19:09:15
4  is fetched at:  26/08/2024 19:09:15
2  is fetched at:  26/08/2024 19:09:15
All 4 pages are fetched!


In [14]:
print(dictContent[3])

{'regno': ['M14871', 'M20744', 'M14543', 'M13152', 'M09658', 'M09658', 'M16126', 'M04179', 'M05553', 'M13988', 'M17359', 'M12266', 'M15670', 'M21203', 'M08886', 'M08886', 'M08886', 'M03640', 'M05069', 'M10650', 'M12781', 'M08126', 'M16127'], 'name': ['區可欣AU, HO YAN TONI', '區學麟AU, HOK LUN', '區漢達AU, HON DA KENNETH', '區嘉輝AU, KA FAI', '區家輝AU, KA FAI RONALD', '區家輝AU, KA FAI RONALD', '區家康AU, KA HONG', '區家裘AU, KA KAU', '區家駒AU, KA KUI', '區嘉敏AU, KA MAN', '歐家南AU, KA NAM', '歐家榮AU, KA WING', '區嘉殷AU, KA YAN ALICE', '歐加兒AU, KA YI CARY', '區啟文AU, KAI MAN', '區啟文AU, KAI MAN', '區啟文AU, KAI MAN', '區鑑東AU, KAM TUNG', '區錦華AU, KAM WAH', '區加明AU, KAR MING', '區建恒AU, KIN HENG CONSTANTINE', '歐健明AU, KIN MING', '區健斌AU, KIN PAN'], 'address': ['', 'DEPARTMENT OF ACCIDENT & EMERGENCY, YAN CHAI HOSPITAL, 7-11 YAN CHAI STREET, TSUEN WAN, N.T.', '', '', '', '', '', '', '', '', '', 'ROOM 146, 1/F, CENTRE FOR HEALTH PROTECTION, 147C ARGYLE STREET, KOWLOON', '香港柴灣樂民道3號東區尤德夫人那打素醫院放射科DEPARTMENT OF RADIOLOGY, G/F, PAMELA YOUDE N

In [15]:
def mergeDict(_dict1: dict, _dict2: dict) -> dict:
    for key, value in _dict2.items():
        if key in _dict1:
            _dict1[key] = _dict1[key] + value
        else:
            _dict1[key] = value
    return _dict1

In [16]:
# merge the dictContent i.e. page->content to dict_all in acsending order 
dict_all = {}

# for dict1 in dictContent:
#     mergeDict(dict_all, dict1)

for key in sorted(dictContent.keys()):
    mergeDict(dict_all, dictContent[key])
    print(f"Key: {key}")

Key: 1
Key: 2
Key: 3
Key: 4


In [17]:
df = pd.DataFrame(dict_all)
#df
df.to_excel("..\\output04\\doctorlistAsync.xlsx")
print("complete.")

complete.


<h2>Working Together</h2>

In [31]:
import nest_asyncio
import asyncio
import aiohttp
from tqdm.notebook import tqdm
from datetime import datetime
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import math

nest_asyncio.apply()

def getNowTime():
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt

# dictionary with {page: content} structure
dictContent = {}

# column text according to headers
def colText(_soup: bs, _headers: str)->list:
    lst = []
    all_text = _soup.findAll("td", attrs={'headers': _headers})    
    for txt in all_text:
        num = txt.get("rowspan")
        if num == None: # no rowspan
            lst.append(txt.text)
            continue        
        if num == "1":
            lst.append(txt.text)
            continue
        if num != "1":
            for j in range(int(num)):
                lst.append(txt.text)
    return lst

def pageContent(_content: str)-> dict:
    if _content == "":
        print("Empty content!")
        return {}
    soup = bs(_content, 'html.parser')
    lstReg = colText(soup, 'reg')
    lstName = colText(soup, 'name')
    lstDate = colText(soup, 'date')
    lstQualification = colText(soup, 'qualification')
    lstQuali = [x for (i, x) in enumerate(lstQualification) if i % 2 == 0]
    lstYear = [x for (i, x) in enumerate(lstQualification) if i % 2 != 0]
    myDict = {'regno': lstReg, 'name': lstName, 'address': lstDate, 'qualification': lstQuali, 'year': lstYear}
    return myDict

async def fetch_page2(_page: int):
    # By default, the limit is set to 100. 
    # If you're making a large number of requests, 
    # you may need to increase this limit e.g. 900 in this scenario
    # async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(limit=2000)) as session:
    async with aiohttp.ClientSession() as session:
        link = f'https://www.mchk.org.hk/tc_chi/list_register/list.php?page={_page}&ipp=20&type=L'
        # set the timeout to be 60 sec to avoid: The semaphore timeout period has expired
        async with session.get(link, timeout=aiohttp.ClientTimeout(total=60)) as response:
        #async with session.get(link) as response:
            content = await response.text()
            dictContent[_page] = pageContent(content)
            print(_page, " is fetched at: ", getNowTime())

def run_fetching2(_lastpage: int):
    if _lastpage > 100: 
        # segmentation of fetching page as 100 per times
        howmanyfetch = math.ceil(_lastpage / 100)
        for i in range(0, howmanyfetch): # 0 to howmanyfetch-1
            print("segment: ", i+1, " starting...")
            initPage = 1 + i * 100
            if (i == howmanyfetch -1):                
                finalPage = _lastpage + 1
                print("final segment: ", "init page: ", initPage, "; final page: ", finalPage-1, ".")
            else:
                finalPage = initPage + 100
            tasks = [ fetch_page2(page) for page in range(initPage, finalPage)]
            asyncio.get_event_loop().run_until_complete(asyncio.gather(*tasks))            
        print(f"All {len(dictContent)} pages are fetched!")
    else:
        tasks = [ fetch_page2(page) for page in range(1, _lastpage+1)]
        asyncio.get_event_loop().run_until_complete(asyncio.gather(*tasks))
        print(f"All {len(dictContent)} pages are fetched!")

def mergeDict(_dict1: dict, _dict2: dict) -> dict:
    for key, value in _dict2.items():
        if key in _dict1:
            _dict1[key] = _dict1[key] + value
        else:
            _dict1[key] = value
    return _dict1

def main():
    run_fetching2(804)
    # merge the dictContent i.e. page->content to dict_all in acsending order 
    dict_all = {}
    for key in sorted(dictContent.keys()):
        mergeDict(dict_all, dictContent[key])
        # print(f"Key: {key}")
    df = pd.DataFrame(dict_all)
    df.to_excel("..\\output04\\doctorlistAsync2.xlsx")
    print("output excel file complete.")

main()

segment:  1  starting...
3  is fetched at:  26/08/2024 20:04:14
47  is fetched at:  26/08/2024 20:04:14
33  is fetched at:  26/08/2024 20:04:14
12  is fetched at:  26/08/2024 20:04:14
41  is fetched at:  26/08/2024 20:04:15
40  is fetched at:  26/08/2024 20:04:15
29  is fetched at:  26/08/2024 20:04:15
27  is fetched at:  26/08/2024 20:04:15
49  is fetched at:  26/08/2024 20:04:15
45  is fetched at:  26/08/2024 20:04:16
15  is fetched at:  26/08/2024 20:04:16
9  is fetched at:  26/08/2024 20:04:16
11  is fetched at:  26/08/2024 20:04:16
10  is fetched at:  26/08/2024 20:04:16
19  is fetched at:  26/08/2024 20:04:16
42  is fetched at:  26/08/2024 20:04:16
39  is fetched at:  26/08/2024 20:04:16
1  is fetched at:  26/08/2024 20:04:16
43  is fetched at:  26/08/2024 20:04:16
48  is fetched at:  26/08/2024 20:04:17
8  is fetched at:  26/08/2024 20:04:17
37  is fetched at:  26/08/2024 20:04:17
4  is fetched at:  26/08/2024 20:04:17
26  is fetched at:  26/08/2024 20:04:18
18  is fetched at:  

<h2>%load the python file</h2>

In [33]:
# %load webtableretrievalasync.py
import nest_asyncio
import asyncio
import aiohttp
from tqdm.notebook import tqdm
from datetime import datetime
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
from datetime import datetime
import math

nest_asyncio.apply()

def getNowTime():
    now = datetime.now()
    # dd/mm/YY H:M:S
    dt = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt

# dictionary with {page: content} structure
dictContent = {}

# column text according to headers
def colText(_soup: bs, _headers: str)->list:
    lst = []
    all_text = _soup.findAll("td", attrs={'headers': _headers})    
    for txt in all_text:
        num = txt.get("rowspan")
        if num == None: # no rowspan
            lst.append(txt.text)
            continue        
        if num == "1":
            lst.append(txt.text)
            continue
        if num != "1":
            for j in range(int(num)):
                lst.append(txt.text)
    return lst

def pageContent(_content: str)-> dict:
    if _content == "":
        print("Empty content!")
        return {}
    soup = bs(_content, 'html.parser')
    lstReg = colText(soup, 'reg')
    lstName = colText(soup, 'name')
    lstDate = colText(soup, 'date')
    lstQualification = colText(soup, 'qualification')
    lstQuali = [x for (i, x) in enumerate(lstQualification) if i % 2 == 0]
    lstYear = [x for (i, x) in enumerate(lstQualification) if i % 2 != 0]
    myDict = {'regno': lstReg, 'name': lstName, 'address': lstDate, 'qualification': lstQuali, 'year': lstYear}
    return myDict

async def fetch_page2(_page: int):
    # By default, the limit is set to 100. 
    # If you're making a large number of requests, 
    # you may need to increase this limit e.g. 900 in this scenario
    # async with aiohttp.ClientSession(connector=aiohttp.TCPConnector(limit=2000)) as session:
    async with aiohttp.ClientSession() as session:
        link = f'https://www.mchk.org.hk/tc_chi/list_register/list.php?page={_page}&ipp=20&type=L'
        # set the timeout to be 60 sec to avoid: The semaphore timeout period has expired
        async with session.get(link, timeout=aiohttp.ClientTimeout(total=60)) as response:
        #async with session.get(link) as response:
            content = await response.text()
            dictContent[_page] = pageContent(content)
            print(_page, " is fetched at: ", getNowTime())

def run_fetching2(_lastpage: int):
    if _lastpage > 100: 
        # segmentation of fetching page as 100 per times
        howmanyfetch = math.ceil(_lastpage / 100)
        for i in range(0, howmanyfetch): # 0 to howmanyfetch-1
            print("segment: ", i+1, " starting...")
            initPage = 1 + i * 100
            if (i == howmanyfetch -1):                
                finalPage = _lastpage + 1
                print("final segment: ", "init page: ", initPage, "; final page: ", finalPage-1, ".")
            else:
                finalPage = initPage + 100
            tasks = [ fetch_page2(page) for page in range(initPage, finalPage)]
            asyncio.get_event_loop().run_until_complete(asyncio.gather(*tasks))            
        print(f"All {len(dictContent)} pages are fetched!")
    else:
        tasks = [ fetch_page2(page) for page in range(1, _lastpage+1)]
        asyncio.get_event_loop().run_until_complete(asyncio.gather(*tasks))
        print(f"All {len(dictContent)} pages are fetched!")

def mergeDict(_dict1: dict, _dict2: dict) -> dict:
    for key, value in _dict2.items():
        if key in _dict1:
            _dict1[key] = _dict1[key] + value
        else:
            _dict1[key] = value
    return _dict1

def main():
    run_fetching2(804)
    # merge the dictContent i.e. page->content to dict_all in acsending order 
    dict_all = {}
    for key in sorted(dictContent.keys()):
        mergeDict(dict_all, dictContent[key])
        # print(f"Key: {key}")
    df = pd.DataFrame(dict_all)
    df.to_excel("..\\output04\\doctorlistAsync2b.xlsx")
    print("output excel file complete.")

main()

segment:  1  starting...
44  is fetched at:  26/08/2024 20:14:47
34  is fetched at:  26/08/2024 20:14:48
49  is fetched at:  26/08/2024 20:14:48
18  is fetched at:  26/08/2024 20:14:48
30  is fetched at:  26/08/2024 20:14:48
45  is fetched at:  26/08/2024 20:14:48
32  is fetched at:  26/08/2024 20:14:48
24  is fetched at:  26/08/2024 20:14:49
27  is fetched at:  26/08/2024 20:14:49
29  is fetched at:  26/08/2024 20:14:49
43  is fetched at:  26/08/2024 20:14:49
15  is fetched at:  26/08/2024 20:14:49
19  is fetched at:  26/08/2024 20:14:49
20  is fetched at:  26/08/2024 20:14:49
1  is fetched at:  26/08/2024 20:14:49
9  is fetched at:  26/08/2024 20:14:50
46  is fetched at:  26/08/2024 20:14:50
33  is fetched at:  26/08/2024 20:14:50
2  is fetched at:  26/08/2024 20:14:50
10  is fetched at:  26/08/2024 20:14:50
14  is fetched at:  26/08/2024 20:14:50
25  is fetched at:  26/08/2024 20:14:50
16  is fetched at:  26/08/2024 20:14:50
39  is fetched at:  26/08/2024 20:14:50
17  is fetched at: